In [1]:
import os
import subprocess
import json
import sys

In [2]:
max_commit = 100 # int(sys.argv[1]) 

# Raw 문자열로 처리하기 위해 문자열 앞에 'r'을 붙임
with open('git-log.json', 'r', encoding='utf-8') as f:
    data = f.read()
    raw_data = r'' + data

# Raw 문자열을 JSON으로 디코드
try:
    json_data = json.loads(raw_data) 
    print(json.dumps(json_data, indent = '\t'))
    
except json.JSONDecodeError as e:
    print(f"JSON 디코드 오류: {e}")

subprocess.run(['mkdir', 'commits'])

[
	{
		"commitHash": "16228f3ccea3c6f1170488e0e268f3601d130f75",
		"parent": "e50487c97a8ae9a3168df89751457ba4cfab4d2a",
		"subject": "build: harden main.yml permissions",
		"sanitized_subject_line": "build-harden-main.yml-permissions",
		"commit_notes": "",
		"body": "Signed-off-by: Alex <aleksandrosansan@gmail.com>",
		"author_name": "Alex",
		"commiter_name": "Marc Philipp",
		"date": "Mon, 1 May 2023 16:25:25 +0200"
	},
	{
		"commitHash": "e50487c97a8ae9a3168df89751457ba4cfab4d2a",
		"parent": "354b72771d70255a860001c6b11786897c1c2fbd 933814efd02b242b804171ac89c74101b0bf1b96",
		"subject": "Merge pull request #1757 from kcooney/eclipse-maven",
		"sanitized_subject_line": "Merge-pull-request-1757-from-kcooney-eclipse-maven",
		"commit_notes": "",
		"body": "Recommend generating Eclipse project files with Maven",
		"author_name": "Kevin Cooney",
		"commiter_name": "GitHub",
		"date": "Mon, 6 Feb 2023 19:44:22 -0800"
	},
	{
		"commitHash": "933814efd02b242b804171ac89c74101b0bf1b96",
	

CompletedProcess(args=['mkdir', 'commits'], returncode=1)

In [3]:
def read_file(file):
    try:
        with open(file, 'r', encoding='utf-8') as f:
            content = f.read()
    except:
        print(f"File Open 오류:")
        content = ''
    return content

def write_file(file_path, content):
    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content)
    except:
        print(f"File Write 오류")

In [4]:
cnt = 0

for commit in json_data:        
    files = []
    commit['changed_file_list'] = []
    
    changed_file_list = subprocess.run(['git', 'show', commit['commitHash'], '--name-only', '--pretty=format:%b'], capture_output=True, text=True)
    if changed_file_list.returncode == 0 and changed_file_list.stdout:
        lines = changed_file_list.stdout.split('\n')
        # Process the rest of the code using the 'lines' variable
    else:
        # Handle the case when the Git command didn't execute successfully
        print("Failed to get the changed file list.")
    
    for line in lines:
        if line.strip() in commit['body']:
            continue
        elif line.strip() != '':
            files.append(line.strip())
                  
    java_files = [file for file in files if file.endswith('.java')]
    java_files = list(filter(lambda file: 'src/test' not in file, java_files))
    
    if java_files != [] :
        diff = subprocess.run(['git', 'show', commit['commitHash'], '-p'], capture_output=True, text=True)
        commit['changed_file_list'] = java_files
        print(files)
        cnt += 1
        print(commit['commitHash'])
        print(cnt, ":", commit['changed_file_list'])
        
        # file_path = os.path.join('commits', f'{cnt}_diff.txt')
        # write_checkout_file(file_path, diff.stdout)

['src/main/java/org/junit/internal/runners/ErrorReportingRunner.java']
65684f6b50cecbfe50936422d3bef8a1e2e5ebff
1 : ['src/main/java/org/junit/internal/runners/ErrorReportingRunner.java']
['src/main/java/org/junit/internal/runners/ErrorReportingRunner.java']
e767bccc69db20942c4f08253c6f957dc12615d3
2 : ['src/main/java/org/junit/internal/runners/ErrorReportingRunner.java']
['src/main/java/org/junit/runner/JUnitCore.java', 'src/test/java/org/junit/runner/MainRunner.java', 'src/test/java/org/junit/tests/running/core/CommandLineTest.java', 'src/test/java/org/junit/tests/running/core/JUnitCoreReturnsCorrectExitCodeTest.java', 'src/test/java/org/junit/tests/running/core/MainRunner.java']
6ba9c4dc1e71e9ceb2e35991e8fc0e4aee69ad6e
3 : ['src/main/java/org/junit/runner/JUnitCore.java']
['src/main/java/org/junit/runner/Result.java']
0eb5ce72848d730da5bd6d42902fdd6a8a42055d
4 : ['src/main/java/org/junit/runner/Result.java']
['src/main/java/org/junit/runners/model/FrameworkMember.java']
43df829aeef59

In [10]:
cnt = 0

for commit in json_data:
    if commit['changed_file_list'] != []:
        cnt += 1
        print(cnt)
    else:
        continue
        
    for file in commit['changed_file_list']:
        print(file)
        sanitized_path = file.replace("/", "_")
        
        subprocess.run(['git', 'checkout', commit['commitHash'], '--', file])
        after_content = read_file(file) 
        # file_path = os.path.join('commits', f'{cnt}_after_{os.path.basename(file)}')
        file_path = os.path.join('commits', f'{cnt}_after_{sanitized_path}')
        write_file(file_path, after_content)
        
        subprocess.run(['git', 'checkout', commit['parent'], '--', file])
        before_content = read_file(file)
        file_path = os.path.join('commits', f'{cnt}_before_{sanitized_path}')
        write_file(file_path, before_content)
        
        subprocess.run(['git', 'checkout', 'HEAD', '--', file])

        hash_path = commit['commitHash']+":"+file
        numOfLine = subprocess.run(['git', 'show', hash_path, "|", "wc", "-l"], shell=True, stdout=subprocess.PIPE, text=True).stdout.rstrip()
        line_option = "-U"+str(numOfLine)
        file_path = os.path.join('commits', f'{cnt}_diff_{sanitized_path}')
        file_option = "--output="+file_path
        parent_hash = commit['commitHash']+"~1"
        subprocess.run(['git', 'diff', line_option, file_option, parent_hash, commit['commitHash'], "--", file])    

1
src/main/java/org/junit/internal/runners/ErrorReportingRunner.java
2
src/main/java/org/junit/internal/runners/ErrorReportingRunner.java
3
src/main/java/org/junit/runner/JUnitCore.java
4
src/main/java/org/junit/runner/Result.java
5
src/main/java/org/junit/runners/model/FrameworkMember.java
6
src/main/java/org/junit/runners/model/FrameworkMember.java
7
src/main/java/org/junit/internal/runners/statements/FailOnTimeout.java
src/main/java/org/junit/runner/JUnitCore.java
8
src/main/java/org/junit/internal/Classes.java
src/main/java/org/junit/runner/FilterFactories.java
src/main/java/org/junit/runner/notification/SynchronizedRunListener.java
9
src/main/java/org/junit/rules/ErrorCollector.java
src/main/java/org/junit/rules/ExpectedException.java


In [14]:
print(cnt)

9


In [5]:
import pandas as pd
df = pd.DataFrame()

In [8]:
df = pd.DataFrame()

cnt = 0
for commit in json_data:
    if commit['changed_file_list'] != []:
        cnt += 1
        print(cnt)
    else:
        continue
    
    print(commit['commitHash'])
    
    for file in commit['changed_file_list']:
        print(file)
        
        sanitized_path = file.replace("/", "_")
        file_path = os.path.join('commits', f'{cnt}_diff_{sanitized_path}')
        diff_content = read_file(file_path)

        file_path = file_path.replace('.java', '_response.txt')
        response_content = read_file(file_path)

        row_data = {
                        "Repository":"junit4",
                        "Hash Code":commit['commitHash'],
                        "Diff Content":diff_content,
                        "4,0 Response":response_content
                    }
        df = df.append(row_data, ignore_index=True)
    
    
df.to_excel("../junit4.xlsx", index=False)

print("데이터가 엑셀 파일에 저장되었습니다.")


1
65684f6b50cecbfe50936422d3bef8a1e2e5ebff
src/main/java/org/junit/internal/runners/ErrorReportingRunner.java


C:\Users\USER\AppData\Local\Temp\ipykernel_2704\1742738584.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_2704\1742738584.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_2704\1742738584.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_2704\1742738584.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\USER\AppDat

2
e767bccc69db20942c4f08253c6f957dc12615d3
src/main/java/org/junit/internal/runners/ErrorReportingRunner.java
3
6ba9c4dc1e71e9ceb2e35991e8fc0e4aee69ad6e
src/main/java/org/junit/runner/JUnitCore.java
4
0eb5ce72848d730da5bd6d42902fdd6a8a42055d
src/main/java/org/junit/runner/Result.java
5
43df829aeef59d65b951a1eff3daf3c6fadbaee1
src/main/java/org/junit/runners/model/FrameworkMember.java
6
45be4aed5aa796e68af5625d73cc8a1c996e3469
src/main/java/org/junit/runners/model/FrameworkMember.java
7
099d5881105422e80fbba2059b4907cb0a3b6f21
src/main/java/org/junit/internal/runners/statements/FailOnTimeout.java
src/main/java/org/junit/runner/JUnitCore.java
8
cda745419600dc6a6b31dc46619288a93c76b6c1
src/main/java/org/junit/internal/Classes.java
src/main/java/org/junit/runner/FilterFactories.java
src/main/java/org/junit/runner/notification/SynchronizedRunListener.java
9
0391e4587c0db1b27fa0132c5cd499951085784b
src/main/java/org/junit/rules/ErrorCollector.java


C:\Users\USER\AppData\Local\Temp\ipykernel_2704\1742738584.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_2704\1742738584.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_2704\1742738584.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_2704\1742738584.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\USER\AppDat

src/main/java/org/junit/rules/ExpectedException.java


C:\Users\USER\AppData\Local\Temp\ipykernel_2704\1742738584.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)


데이터가 엑셀 파일에 저장되었습니다.


In [34]:
try:
    df = pd.read_excel("../all_commits.xlsx")
except FileNotFoundError:
    df = pd.DataFrame()

cnt = 0
for commit in json_data:
    if commit['changed_file_list'] != []:
        cnt += 1
        print(cnt)
    else:
        continue
        
    for file in commit['changed_file_list']:
        print(file)
        
        file_path = os.path.join('commits', f'{cnt}_a_{os.path.basename(file)}')
        after_content = read_checkout_file(file_path)

        file_path = file_path.replace('.java', '_response.txt')
        response_content = read_checkout_file(file_path)
        
        row_data = {
                    "Repository":"junit4",
                    "Hash Code":commit['commitHash'],
                    "File Name":file,
                    "Content":after_content,
                    "Response":response_content
                    }
        df = df.append(row_data, ignore_index=True)
    
df.to_excel("../junit4_diff.xlsx", index=False)

print("데이터가 엑셀 파일에 저장되었습니다.")


1
src/main/java/org/junit/internal/runners/ErrorReportingRunner.java


C:\Users\USER\AppData\Local\Temp\ipykernel_10096\144254068.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_10096\144254068.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_10096\144254068.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_10096\144254068.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\USER\AppDat

2
src/main/java/org/junit/internal/runners/ErrorReportingRunner.java
3
src/main/java/org/junit/runner/JUnitCore.java
4
src/main/java/org/junit/runner/Result.java
5
src/main/java/org/junit/runners/model/FrameworkMember.java
6

C:\Users\USER\AppData\Local\Temp\ipykernel_10096\144254068.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_10096\144254068.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_10096\144254068.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_10096\144254068.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)



src/main/java/org/junit/runners/model/FrameworkMember.java
7
src/main/java/org/junit/internal/runners/statements/FailOnTimeout.java
src/main/java/org/junit/runner/JUnitCore.java
8
src/main/java/org/junit/internal/Classes.java
src/main/java/org/junit/runner/FilterFactories.java


C:\Users\USER\AppData\Local\Temp\ipykernel_10096\144254068.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_10096\144254068.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_10096\144254068.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_10096\144254068.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)


src/main/java/org/junit/runner/notification/SynchronizedRunListener.java
9
src/main/java/org/junit/rules/ErrorCollector.java
src/main/java/org/junit/rules/ExpectedException.java
데이터가 엑셀 파일에 저장되었습니다.
